In [1]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np 
import pandas as pd 
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib

target_name = 'target'

import joblib

In [2]:
data_dir = '../input/optiver-realized-volatility-prediction/'


def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap


def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap


def log_return(series):
    return np.log(series).diff()


def realized_volatility(series):
    return np.sqrt(np.sum(series**2))


def count_unique(series):
    return len(np.unique(series))


def read_test():
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    return test


def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    
    df['wap_mean'] = (df['wap1'] + df['wap2']) / 2
    
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    df['bas'] = (df[['ask_price1', 'ask_price2']].min(axis = 1)/ df[['bid_price1', 'bid_price2']].max(axis = 1) - 1) 
    df['h_spread_l1'] = df['ask_price1'] - df['bid_price1']
    df['h_spread_l2'] = df['ask_price2'] - df['bid_price2']
    df['log_return_bid_price1'] = np.log(df['bid_price1'].pct_change() + 1)
    df['log_return_ask_price1'] = np.log(df['ask_price1'].pct_change() + 1)
    df['log_return_bid_size1'] = np.log(df['bid_size1'].pct_change() + 1)
    df['log_return_ask_size1'] = np.log(df['ask_size1'].pct_change() + 1)
    df['log_ask_1_div_bid_1'] = np.log(df['ask_price1'] / df['bid_price1'])
    df['log_ask_1_div_bid_1_size'] = np.log(df['ask_size1'] / df['bid_size1'])
    df['log_return_bid_price2'] = np.log(df['bid_price2'].pct_change() + 1)
    df['log_return_ask_price2'] = np.log(df['ask_price2'].pct_change() + 1)
    df['log_return_bid_size2'] = np.log(df['bid_size2'].pct_change() + 1)
    df['log_return_ask_size2'] = np.log(df['ask_size2'].pct_change() + 1)
    df['log_ask_2_div_bid_2'] = np.log(df['ask_price2'] / df['bid_price2'])
    df['log_ask_2_div_bid_2_size'] = np.log(df['ask_size2'] / df['bid_size2'])
    
    
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    
    
    create_feature_dict = {
        'wap1': [np.mean,np.max,np.std],
        'wap2': [np.mean,np.max,np.std],

        'log_return1': [realized_volatility,np.sum],
        'log_return2': [realized_volatility,np.sum], 
        'log_return_bid_price1':[realized_volatility,np.sum], 
        'log_return_ask_price1':[realized_volatility,np.sum],
        'log_return_bid_size1':[realized_volatility,np.sum], 
        'log_return_ask_size1':[realized_volatility,np.sum], 
        'log_ask_1_div_bid_1':[realized_volatility,np.sum], 
        'log_ask_1_div_bid_1_size':[realized_volatility,np.sum], 
        'log_return_bid_price2':[realized_volatility,np.sum], 
        'log_return_ask_price2':[realized_volatility,np.sum], 
        'log_return_bid_size2':[realized_volatility,np.sum], 
        'log_return_ask_size2':[realized_volatility,np.sum], 
        'log_ask_2_div_bid_2':[realized_volatility,np.sum], 
        'log_ask_2_div_bid_2_size':[realized_volatility,np.sum], 
 
        'wap_balance': [np.mean,np.max,np.std,np.sum], 
        'wap_mean': [np.mean,np.max,np.std,np.sum],
        'price_spread':[np.mean,np.max,np.std,np.sum],
        'bid_spread':[np.mean,np.max,np.std,np.sum],
        'ask_spread':[np.mean,np.max,np.std,np.sum],
        'total_volume':[np.mean,np.max,np.std,np.sum],
        'volume_imbalance':[np.mean,np.max,np.std,np.sum],
        'bas':[np.mean,np.max,np.std,np.sum],
        'h_spread_l1':[np.mean,np.max,np.std,np.sum],
        'h_spread_l2':[np.mean,np.max,np.std,np.sum],
        'price_spread2':[np.mean,np.max,np.std,np.sum],
        "bid_ask_spread":[np.mean,np.max,np.std,np.sum]}
    
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)

    
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200'], axis = 1, inplace = True)
    
    
    
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature


def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max],
    }
    
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200','time_id'], axis = 1, inplace = True)
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature


def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']
    
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    

def preprocessor(list_stock_ids, is_train = True):
    
   
    def for_joblib(stock_id):
        
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        
        return df_tmp
    
    
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    
    df = pd.concat(df, ignore_index = True)
    return df


def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

test = read_test()


test_stock_ids = test['stock_id'].unique()

test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')


test = get_time_stock(test)

test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.25/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.75/ test['trade_order_count_sum'] )

test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.3s finished


In [3]:
from numpy.random import seed
seed(21)
import tensorflow as tf
tf.random.set_seed(21)
from tensorflow.keras import backend as K
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow import keras
def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')
colNames = list(test)
colNames.remove('time_id')
colNames.remove('row_id')
colNames.remove('stock_id')
test.replace([np.inf, -np.inf], np.nan,inplace=True)
from pickle import load,dump
for col in colNames:
    qt=load(open(f'/kaggle/input/ng-wwdd/qt_{col}.pkl', 'rb'))
    test[col] = qt.transform(test[[col]])  

2021-09-27 11:26:03.718224: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
from sklearn.cluster import KMeans
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

label=[1, 0, 4, 2, 1, 1, 2, 4, 6, 2, 1, 0, 4, 4, 1, 1, 1, 2, 4, 4, 4, 0, 1, 1, 3, 1, 1, 4, 3, 4, 3, 4, 4, 1, 3, 3, 4,
       3, 4, 1, 4, 1, 4, 4, 1, 0, 4, 4, 1, 0, 0, 3, 3, 3, 2, 0, 2, 4, 1, 4, 4, 1, 4, 1, 0, 3, 3, 0, 3, 0, 6, 5, 3, 3,
       0, 1, 2, 0, 3, 3, 3, 4, 1, 1, 0, 2, 3, 3, 1, 0, 1, 4, 4, 4, 4, 4, 1, 3, 1, 0, 1, 4, 1, 0, 1, 4, 1, 0, 4, 0, 4,
       0]

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(label==n*np.ones(len(label)))) if x > 0] )
    
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1  
mat2 = pd.concat(matTest).reset_index()
mat1=pd.read_csv('/kaggle/input/ng-wwdd/mat1.csv')
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_2c1',
     'total_volume_mean_0c1',
     'total_volume_mean_1c1', 
     'total_volume_mean_3c1',
     'total_volume_mean_4c1', 
     'total_volume_mean_2c1',
     'trade_size_mean_0c1',
     'trade_size_mean_1c1', 
     'trade_size_mean_3c1',
     'trade_size_mean_4c1', 
     'trade_size_mean_2c1',
     'trade_order_count_mean_0c1',
     'trade_order_count_mean_1c1',
     'trade_order_count_mean_3c1',
     'trade_order_count_mean_4c1',
     'trade_order_count_mean_2c1',      
     'price_spread_mean_0c1',
     'price_spread_mean_1c1',
     'price_spread_mean_3c1',
     'price_spread_mean_4c1',
     'price_spread_mean_2c1',   
     'bid_spread_mean_0c1',
     'bid_spread_mean_1c1',
     'bid_spread_mean_3c1',
     'bid_spread_mean_4c1',
     'bid_spread_mean_2c1',       
     'ask_spread_mean_0c1',
     'ask_spread_mean_1c1',
     'ask_spread_mean_3c1',
     'ask_spread_mean_4c1',
     'ask_spread_mean_2c1',   
     'volume_imbalance_mean_0c1',
     'volume_imbalance_mean_1c1',
     'volume_imbalance_mean_3c1',
     'volume_imbalance_mean_4c1',
     'volume_imbalance_mean_2c1',       
     'bid_ask_spread_mean_0c1',
     'bid_ask_spread_mean_1c1',
     'bid_ask_spread_mean_3c1',
     'bid_ask_spread_mean_4c1',
     'bid_ask_spread_mean_2c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_2c1'] 
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

[1, 11, 22, 50, 55, 56, 62, 73, 76, 78, 84, 87, 96, 101, 112, 116, 122, 124, 126]
[0, 4, 5, 10, 15, 16, 17, 23, 26, 28, 29, 36, 42, 44, 48, 53, 66, 69, 72, 85, 94, 95, 100, 102, 109, 111, 113, 115, 118, 120]
[3, 6, 9, 18, 61, 63, 86, 97]
[27, 31, 33, 37, 38, 40, 58, 59, 60, 74, 75, 77, 82, 83, 88, 89, 90, 98, 99, 110]
[2, 7, 13, 14, 19, 20, 21, 30, 32, 34, 35, 39, 41, 43, 46, 47, 51, 52, 64, 67, 68, 70, 93, 103, 104, 105, 107, 108, 114, 119, 123, 125]
[81]
[8, 80]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


In [5]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
get_custom_objects().update({'swish': Activation(swish)})
hidden_units = (128,64,32)
stock_embedding_size = 24

train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
cat_data =train['stock_id']
del train
gc.collect()

def base_model():
    
    
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(538,), name='num_data')


    
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='swish')(out)
        

    

    
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model

In [6]:
features_to_consider = list(test)
features_to_consider.remove('time_id')
features_to_consider.remove('row_id')
f_mean = np.load('/kaggle/input/ng-wwdd/f_mean.npy')
for i in range(len(features_to_consider)):
    test[features_to_consider[i]] = test[features_to_consider[i]].fillna(f_mean[i])

In [7]:
test.head()

,stock_id,time_id,row_id,wap1_mean,wap1_amax,wap1_std,wap2_mean,wap2_amax,wap2_std,log_return1_realized_volatility,...,bid_ask_spread_mean_0c1,bid_ask_spread_mean_1c1,bid_ask_spread_mean_3c1,bid_ask_spread_mean_4c1,bid_ask_spread_mean_2c1,size_tau2_0c1,size_tau2_1c1,size_tau2_3c1,size_tau2_4c1,size_tau2_2c1
0,0,4,0-4,0.250325,-0.543764,-2.429043,0.335747,-0.527097,-2.723074,-3.421779,...,-0.279017,0.823808,0.519245,-0.030445,0.358564,-0.086921,3.161571,0.590543,-0.161021,-0.106622
1,0,32,0-32,-0.001311,-0.000514,0.000281,-0.001294,-0.000760,0.000382,0.001459,...,-0.279017,-0.260539,0.519245,-0.030445,0.358564,-0.086921,-0.085898,0.590543,-0.161021,-0.106622
2,0,34,0-34,-0.001311,-0.000514,0.000281,-0.001294,-0.000760,0.000382,0.001459,...,-0.279017,-0.260539,0.519245,-0.030445,0.358564,-0.086921,-0.085898,0.590543,-0.161021,-0.106622


In [8]:
n_folds=5
test_preds = np.zeros(len(test))

for n_count in range(n_folds):
    
    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.004), 
        loss=root_mean_squared_per_error
    )
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    model.load_weights(f'../input/ng-wwdd/n2_model_{n_count}.hdf5')

    scaler=load(open(f'/kaggle/input/ng-wwdd/scaler_{n_count}.pkl', 'rb'))
    tt =scaler.transform(test[features_to_consider].values)
    test_preds += model.predict([test['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)
       
    features_to_consider.append('stock_id')
    del model
    gc.collect()

2021-09-27 11:26:20.289396: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-27 11:26:20.292284: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-27 11:26:20.335475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-27 11:26:20.336092: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-27 11:26:20.336162: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-27 11:26:20.363517: I tensorflow/stream_executor/platform/def

In [9]:
ng_ffgg1=test_preds/n_folds

In [10]:
!pip -q install ../input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.matlib

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

from scipy import stats
from scipy.stats import norm
from joblib import Parallel, delayed

import shutil
import glob

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts




plt.style.use('ggplot')
orange_black = [
    '#fdc029', '#df861d', '#FF6347', '#aa3d01', '#a30e15', '#800000', '#171820'
]
plt.rcParams['figure.figsize'] = (16,9)
plt.rcParams["figure.facecolor"] = '#FFFACD'
plt.rcParams["axes.facecolor"] = '#FFFFE0'
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = orange_black[3]
plt.rcParams["grid.alpha"] = 0.5
plt.rcParams["grid.linestyle"] = '--'


import warnings
warnings.filterwarnings("ignore")

In [12]:
data_dir = '../input/optiver-realized-volatility-prediction/'


def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap


def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap


def log_return(series):
    return np.log(series).diff()


def realized_volatility(series):
    return np.sqrt(np.sum(series**2))


def count_unique(series):
    return len(np.unique(series))


def read_train_test():
    
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    
    
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    
    return test


def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    
    df['wap_mean'] = (df['wap1'] + df['wap2']) / 2
    
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    df['bas'] = (df[['ask_price1', 'ask_price2']].min(axis = 1)/ df[['bid_price1', 'bid_price2']].max(axis = 1) - 1) 
    df['h_spread_l1'] = df['ask_price1'] - df['bid_price1']
    df['h_spread_l2'] = df['ask_price2'] - df['bid_price2']
    df['log_return_bid_price1'] = np.log(df['bid_price1'].pct_change() + 1)
    df['log_return_ask_price1'] = np.log(df['ask_price1'].pct_change() + 1)
    df['log_return_bid_size1'] = np.log(df['bid_size1'].pct_change() + 1)
    df['log_return_ask_size1'] = np.log(df['ask_size1'].pct_change() + 1)
    df['log_ask_1_div_bid_1'] = np.log(df['ask_price1'] / df['bid_price1'])
    df['log_ask_1_div_bid_1_size'] = np.log(df['ask_size1'] / df['bid_size1'])
    df['log_return_bid_price2'] = np.log(df['bid_price2'].pct_change() + 1)
    df['log_return_ask_price2'] = np.log(df['ask_price2'].pct_change() + 1)
    df['log_return_bid_size2'] = np.log(df['bid_size2'].pct_change() + 1)
    df['log_return_ask_size2'] = np.log(df['ask_size2'].pct_change() + 1)
    df['log_ask_2_div_bid_2'] = np.log(df['ask_price2'] / df['bid_price2'])
    df['log_ask_2_div_bid_2_size'] = np.log(df['ask_size2'] / df['bid_size2'])
    
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    
    
    
    create_feature_dict = {
        'wap1': [np.mean,np.max,np.std],
        'wap2': [np.mean,np.max,np.std],

        'log_return1': [realized_volatility,np.sum],
        'log_return2': [realized_volatility,np.sum], 
        'log_return_bid_price1':[realized_volatility,np.sum], 
        'log_return_ask_price1':[realized_volatility,np.sum],
        'log_return_bid_size1':[realized_volatility,np.sum], 
        'log_return_ask_size1':[realized_volatility,np.sum], 
        'log_ask_1_div_bid_1':[realized_volatility,np.sum], 
        'log_ask_1_div_bid_1_size':[realized_volatility,np.sum], 
        'log_return_bid_price2':[realized_volatility,np.sum], 
        'log_return_ask_price2':[realized_volatility,np.sum], 
        'log_return_bid_size2':[realized_volatility,np.sum], 
        'log_return_ask_size2':[realized_volatility,np.sum], 
        'log_ask_2_div_bid_2':[realized_volatility,np.sum], 
        'log_ask_2_div_bid_2_size':[realized_volatility,np.sum], 
 
        'wap_balance': [np.mean,np.max,np.std,np.sum], 
        'wap_mean': [np.mean,np.max,np.std,np.sum],
        'price_spread':[np.mean,np.max,np.std,np.sum],
        'bid_spread':[np.mean,np.max,np.std,np.sum],
        'ask_spread':[np.mean,np.max,np.std,np.sum],
        'total_volume':[np.mean,np.max,np.std,np.sum],
        'volume_imbalance':[np.mean,np.max,np.std,np.sum],
        'bas':[np.mean,np.max,np.std,np.sum],
        'h_spread_l1':[np.mean,np.max,np.std,np.sum],
        'h_spread_l2':[np.mean,np.max,np.std,np.sum],
        'price_spread2':[np.mean,np.max,np.std,np.sum],
        "bid_ask_spread":[np.mean,np.max,np.std,np.sum]
    }
    
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    
    
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')

    
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200'], axis = 1, inplace = True)
    
    
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max],
    }
    
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')

    
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200','time_id'], axis = 1, inplace = True)
    
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature


def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']


    
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    

def preprocessor(list_stock_ids, is_train = True):
    
    
    def for_joblib(stock_id):
        
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        
        return df_tmp
    
    
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    
    df = pd.concat(df, ignore_index = True)
    return df


def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

test = read_train_test()


test_stock_ids = test['stock_id'].unique()

test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')


test = get_time_stock(test)

test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )


test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.7s finished


In [13]:
from sklearn.cluster import KMeans


train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

label=[1, 0, 4, 2, 1, 1, 2, 4, 6, 2, 1, 0, 4, 4, 1, 1, 1, 2, 4, 4, 4, 0, 1, 1, 3, 1, 1, 4, 3, 4, 3, 4, 4, 1, 3, 3, 4,
       3, 4, 1, 4, 1, 4, 4, 1, 0, 4, 4, 1, 0, 0, 3, 3, 3, 2, 0, 2, 4, 1, 4, 4, 1, 4, 1, 0, 3, 3, 0, 3, 0, 6, 5, 3, 3,
       0, 1, 2, 0, 3, 3, 3, 4, 1, 1, 0, 2, 3, 3, 1, 0, 1, 4, 4, 4, 4, 4, 1, 3, 1, 0, 1, 4, 1, 0, 1, 4, 1, 0, 4, 0, 4,
       0]

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(label==n*np.ones(len(label)))) if x > 0] )
    


matTest = []

n = 0
for ind in l:
    print(ind)
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1

mat2 = pd.concat(matTest).reset_index()
mat1=pd.read_csv('/kaggle/input/tg-wwdd/mat1.csv')
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_2c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_2c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_2c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_2c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_2c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_2c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_2c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_2c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_2c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_2c1'] 
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

[1, 11, 22, 50, 55, 56, 62, 73, 76, 78, 84, 87, 96, 101, 112, 116, 122, 124, 126]
[0, 4, 5, 10, 15, 16, 17, 23, 26, 28, 29, 36, 42, 44, 48, 53, 66, 69, 72, 85, 94, 95, 100, 102, 109, 111, 113, 115, 118, 120]
[3, 6, 9, 18, 61, 63, 86, 97]
[27, 31, 33, 37, 38, 40, 58, 59, 60, 74, 75, 77, 82, 83, 88, 89, 90, 98, 99, 110]
[2, 7, 13, 14, 19, 20, 21, 30, 32, 34, 35, 39, 41, 43, 46, 47, 51, 52, 64, 67, 68, 70, 93, 103, 104, 105, 107, 108, 114, 119, 123, 125]
[81]
[8, 80]


In [14]:
features = [col for col in test.columns.tolist() if col not in ['time_id','row_id']]
f_mean = np.load('/kaggle/input/tg-wwdd/f_mean.npy')
for i in range(len(features)):
    test[features[i]] = test[features[i]].fillna(f_mean[i])

In [15]:
X_test=test.copy()
X_test.drop(['time_id','row_id'], axis=1,inplace=True)
def rmspe(y_true, y_pred):
    
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

class RMSPE(Metric):
    def __init__(self):
        self._name = "rmspe"
        self._maximize = False

    def __call__(self, y_true, y_score):
        
        return np.sqrt(np.mean(np.square((y_true - y_score) / y_true)))
    


def RMSPELoss(y_pred, y_true):
    return torch.sqrt(torch.mean( ((y_true - y_pred) / y_true) ** 2 )).clone()
from pickle import load,dump

for col in features:
    if  col == 'stock_id':
        l_enc=load(open('/kaggle/input/tg-wwdd/l_enc.pkl', 'rb'))
        X_test[col] = l_enc.transform(X_test[col].values)
    else:
        scaler=load(open(f'/kaggle/input/tg-wwdd/scaler_{col}.pkl', 'rb'))
        X_test[col] = scaler.transform(X_test[col].values.reshape(-1, 1))
cat_idxs_df=pd.read_csv('/kaggle/input/tg-wwdd/cat_idxs_df.csv')
cat_dims_df=pd.read_csv('/kaggle/input/tg-wwdd/cat_dims_df.csv')
cat_idxs=cat_idxs_df['cat_idxs']
cat_dims=cat_dims_df['cat_dims']
tabnet_params = dict(
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=1,
    n_d = 13,
    n_a = 13,
    n_steps = 1,
    gamma = 2,
    n_independent = 2,
    n_shared = 2,
    lambda_sparse = 0,
    optimizer_fn = Adam,
    optimizer_params = dict(lr = (2e-2)),
    mask_type = "entmax",
    scheduler_params = dict(T_0=200, T_mult=1, eta_min=1e-4, last_epoch=-1, verbose=False),
    scheduler_fn = CosineAnnealingWarmRestarts,
    seed = 60,
    verbose = 10
    
)

In [16]:
X_test.head()

,stock_id,wap1_mean,wap1_amax,wap1_std,wap2_mean,wap2_amax,wap2_std,log_return1_realized_volatility,log_return1_sum,log_return2_realized_volatility,...,bid_ask_spread_sum_0c1,bid_ask_spread_sum_1c1,bid_ask_spread_sum_3c1,bid_ask_spread_sum_4c1,bid_ask_spread_sum_2c1,size_tau2_0c1,size_tau2_1c1,size_tau2_3c1,size_tau2_4c1,size_tau2_2c1
0,0,1.191565e-01,-4.375152e-01,-8.941496e-01,1.623597e-01,-4.365765e-01,-9.351345e-01,-1.098426e+00,8.241560e-02,-1.105520e+00,...,1.408291e-13,-2.253557e+00,1.374878e-13,6.954657e-14,2.642473e-13,-2.844108e-13,2.003125e+01,-5.454171e-13,-1.019213e-12,-1.264623e-13
1,0,-2.042257e-12,1.225712e-12,3.505232e-15,9.872247e-13,-5.769340e-14,9.161438e-15,9.433118e-15,4.194278e-17,9.664863e-15,...,1.408291e-13,-3.196314e-13,1.374878e-13,6.954657e-14,2.642473e-13,-2.844108e-13,6.856557e-13,-5.454171e-13,-1.019213e-12,-1.264623e-13
2,0,-2.042257e-12,1.225712e-12,3.505232e-15,9.872247e-13,-5.769340e-14,9.161438e-15,9.433118e-15,4.194278e-17,9.664863e-15,...,1.408291e-13,-3.196314e-13,1.374878e-13,6.954657e-14,2.642473e-13,-2.844108e-13,6.856557e-13,-5.454171e-13,-1.019213e-12,-1.264623e-13


In [17]:
import os
pathB = "../input/tg-wwdd/"
modelpath = [os.path.join(pathB,s) for s in os.listdir(pathB) if ("zip" in s)]

In [18]:
test_predictions=[]
for path in modelpath:
    clf =  TabNetRegressor(**tabnet_params)
    clf.load_model(path)
    test_predictions.append(clf.predict(X_test.values).squeeze(-1))

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda


In [19]:
tg_ffgg1 =  np.mean(test_predictions,axis=0)

In [20]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np 
import pandas as pd 
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib

target_name = 'target'

import joblib

In [21]:
data_dir = '../input/optiver-realized-volatility-prediction/'


def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap


def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap


def log_return(series):
    return np.log(series).diff()


def realized_volatility(series):
    return np.sqrt(np.sum(series**2))


def count_unique(series):
    return len(np.unique(series))


def read_test():
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    return test


def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)
    
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)
    
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    
    create_feature_dict = {
        'wap1': [np.sum, np.std],
        'wap2': [np.sum, np.std],
        'wap3': [np.sum, np.std],
        'wap4': [np.sum, np.std],
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'wap_balance': [np.sum, np.max],
        'price_spread':[np.sum, np.max],
        'price_spread2':[np.sum, np.max],
        'bid_spread':[np.sum, np.max],
        'ask_spread':[np.sum, np.max],
        'total_volume':[np.sum, np.max],
        'volume_imbalance':[np.sum, np.max],
        "bid_ask_spread":[np.sum,  np.max],
    }
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
    }
    
    
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

    
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    
    
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature


def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount']=df['price']*df['size']
    
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
    }
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
    }
    
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature


def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']


    
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    

def preprocessor(list_stock_ids, is_train = True):
    
    
    def for_joblib(stock_id):
        
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        
        return df_tmp
    
    
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    
    df = pd.concat(df, ignore_index = True)
    return df


def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

test = read_test()


test_stock_ids = test['stock_id'].unique()

test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')


test = get_time_stock(test)

test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )


test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [22]:
from sklearn.cluster import KMeans
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

label=[1, 0, 4, 2, 1, 1, 2, 4, 6, 2, 1, 0, 4, 4, 1, 1, 1, 2, 4, 4, 4, 0, 1, 1, 3, 1, 1, 4, 3, 4, 3, 4, 4, 1, 3, 3, 4,
       3, 4, 1, 4, 1, 4, 4, 1, 0, 4, 4, 1, 0, 0, 3, 3, 3, 2, 0, 2, 4, 1, 4, 4, 1, 4, 1, 0, 3, 3, 0, 3, 0, 6, 5, 3, 3,
       0, 1, 2, 0, 3, 3, 3, 4, 1, 1, 0, 2, 3, 3, 1, 0, 1, 4, 4, 4, 4, 4, 1, 3, 1, 0, 1, 4, 1, 0, 1, 4, 1, 0, 4, 0, 4,
       0]

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(label==n*np.ones(len(label)))) if x > 0] )
    
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1  
mat2 = pd.concat(matTest).reset_index()
mat1=pd.read_csv('/kaggle/input/lg-wwdd/mat1.csv')
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_2c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_2c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_2c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_2c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_2c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_2c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_2c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_2c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_2c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_2c1'] 
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

[1, 11, 22, 50, 55, 56, 62, 73, 76, 78, 84, 87, 96, 101, 112, 116, 122, 124, 126]
[0, 4, 5, 10, 15, 16, 17, 23, 26, 28, 29, 36, 42, 44, 48, 53, 66, 69, 72, 85, 94, 95, 100, 102, 109, 111, 113, 115, 118, 120]
[3, 6, 9, 18, 61, 63, 86, 97]
[27, 31, 33, 37, 38, 40, 58, 59, 60, 74, 75, 77, 82, 83, 88, 89, 90, 98, 99, 110]
[2, 7, 13, 14, 19, 20, 21, 30, 32, 34, 35, 39, 41, 43, 46, 47, 51, 52, 64, 67, 68, 70, 93, 103, 104, 105, 107, 108, 114, 119, 123, 125]
[81]
[8, 80]


In [23]:
test.head()

,stock_id,time_id,row_id,wap1_sum,wap1_std,wap2_sum,wap2_std,wap3_sum,wap3_std,wap4_sum,...,bid_ask_spread_sum_0c1,bid_ask_spread_sum_1c1,bid_ask_spread_sum_3c1,bid_ask_spread_sum_4c1,bid_ask_spread_sum_2c1,size_tau2_0c1,size_tau2_1c1,size_tau2_3c1,size_tau2_4c1,size_tau2_2c1
0,0,4,0-4,3.001215,0.00017,3.00165,0.000153,3.000752,0.000142,2.999481,...,NaN,0.001524,NaN,NaN,NaN,NaN,0.301511,NaN,NaN,NaN
1,0,32,0-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,34,0-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
from sklearn.model_selection import KFold
import lightgbm as lgb
import glob
import joblib
import lightgbm as lgb
x_test = test.drop(['row_id', 'time_id'], axis = 1)
x_test['stock_id'] = x_test['stock_id'].astype(int)

In [25]:
MODEL_DIR =  "../input/lg-wwdd"
lg_ffgg1 = np.zeros(len(x_test))
files = glob.glob(f'{MODEL_DIR}/*model*.pkl')
for i, f in enumerate(files):
    model = joblib.load(f)
    lg_ffgg1 += model.predict(x_test)
lg_ffgg1=lg_ffgg1/5

In [26]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np 
import pandas as pd 
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib

target_name = 'target'

import joblib

In [27]:
data_dir = '../input/optiver-realized-volatility-prediction/'


def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap


def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap


def log_return(series):
    return np.log(series).diff()


def realized_volatility(series):
    return np.sqrt(np.sum(series**2))


def count_unique(series):
    return len(np.unique(series))


def read_test():
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
  
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    return test


def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    
    df['wap_mean'] = (df['wap1'] + df['wap2']) / 2

    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    df['bas'] = (df[['ask_price1', 'ask_price2']].min(axis = 1)/ df[['bid_price1', 'bid_price2']].max(axis = 1) - 1) 
    df['h_spread_l1'] = df['ask_price1'] - df['bid_price1']
    df['h_spread_l2'] = df['ask_price2'] - df['bid_price2']
    df['log_return_bid_price1'] = np.log(df['bid_price1'].pct_change() + 1)
    df['log_return_ask_price1'] = np.log(df['ask_price1'].pct_change() + 1)
    df['log_return_bid_size1'] = np.log(df['bid_size1'].pct_change() + 1)
    df['log_return_ask_size1'] = np.log(df['ask_size1'].pct_change() + 1)
    df['log_ask_1_div_bid_1'] = np.log(df['ask_price1'] / df['bid_price1'])
    df['log_ask_1_div_bid_1_size'] = np.log(df['ask_size1'] / df['bid_size1'])
    df['log_return_bid_price2'] = np.log(df['bid_price2'].pct_change() + 1)
    df['log_return_ask_price2'] = np.log(df['ask_price2'].pct_change() + 1)
    df['log_return_bid_size2'] = np.log(df['bid_size2'].pct_change() + 1)
    df['log_return_ask_size2'] = np.log(df['ask_size2'].pct_change() + 1)
    df['log_ask_2_div_bid_2'] = np.log(df['ask_price2'] / df['bid_price2'])
    df['log_ask_2_div_bid_2_size'] = np.log(df['ask_size2'] / df['bid_size2'])
    
   
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    
   
    create_feature_dict = {'wap1': [np.mean,np.max,np.std],
        'wap2': [np.mean,np.max,np.std],

        'log_return1': [realized_volatility,np.sum],
        'log_return2': [realized_volatility,np.sum], 
        'log_return_bid_price1':[realized_volatility,np.sum], 
        'log_return_ask_price1':[realized_volatility,np.sum],
        'log_return_bid_size1':[realized_volatility,np.sum], 
        'log_return_ask_size1':[realized_volatility,np.sum], 
        'log_ask_1_div_bid_1':[realized_volatility,np.sum], 
        'log_ask_1_div_bid_1_size':[realized_volatility,np.sum], 
        'log_return_bid_price2':[realized_volatility,np.sum], 
        'log_return_ask_price2':[realized_volatility,np.sum], 
        'log_return_bid_size2':[realized_volatility,np.sum], 
        'log_return_ask_size2':[realized_volatility,np.sum], 
        'log_ask_2_div_bid_2':[realized_volatility,np.sum], 
        'log_ask_2_div_bid_2_size':[realized_volatility,np.sum], 
 
        'wap_balance': [np.mean,np.max,np.std,np.sum], 
        'wap_mean': [np.mean,np.max,np.std,np.sum],
        'price_spread':[np.mean,np.max,np.std,np.sum],
        'bid_spread':[np.mean,np.max,np.std,np.sum],
        'ask_spread':[np.mean,np.max,np.std,np.sum],
        'total_volume':[np.mean,np.max,np.std,np.sum],
        'volume_imbalance':[np.mean,np.max,np.std,np.sum],
        'bas':[np.mean,np.max,np.std,np.sum],
        'h_spread_l1':[np.mean,np.max,np.std,np.sum],
        'h_spread_l2':[np.mean,np.max,np.std,np.sum],
        'price_spread2':[np.mean,np.max,np.std,np.sum],
        "bid_ask_spread":[np.mean,np.max,np.std,np.sum],}
    
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
       
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
      
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
 
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)

   
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200'], axis = 1, inplace = True)
    
    
  
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature


def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max],
    }
    
  
    def get_stats_window(seconds_in_bucket, add_suffix = False):
     
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
       
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
      
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

   
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
     
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
  
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
   
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200','time_id'], axis = 1, inplace = True)
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature


def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']
   
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

  
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
   
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
   
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    

def preprocessor(list_stock_ids, is_train = True):
    
   
    def for_joblib(stock_id):
     
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
     
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
       
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
      
        return df_tmp
    
  
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
   
    df = pd.concat(df, ignore_index = True)
    return df


def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

test = read_test()


test_stock_ids = test['stock_id'].unique()

test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')


test = get_time_stock(test)

test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.25/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.75/ test['trade_order_count_sum'] )

test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.7s finished


In [28]:
from numpy.random import seed
seed(60)
import tensorflow as tf
tf.random.set_seed(60)
from tensorflow.keras import backend as K
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow import keras
def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')
colNames = list(test)
colNames.remove('time_id')
colNames.remove('row_id')
colNames.remove('stock_id')
test.replace([np.inf, -np.inf], np.nan,inplace=True)
from pickle import load,dump
for col in colNames:
    qt=load(open(f'/kaggle/input/nk-wwdd/qt_{col}.pkl', 'rb'))
    test[col] = qt.transform(test[[col]])  

In [29]:
from sklearn.cluster import KMeans
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

label=[1, 0, 4, 2, 1, 1, 2, 4, 6, 2, 1, 0, 4, 4, 1, 1, 1, 2, 4, 4, 4, 0, 1, 1, 3, 1, 1, 4, 3, 4, 3, 4, 4, 1, 3, 3, 4,
 3, 4, 1, 4, 1, 4, 4, 1, 0, 4, 4, 1, 0, 0, 3, 3, 3, 2, 0, 2, 4, 1, 4, 4, 1, 4, 1, 0, 3, 3, 0, 3, 0, 6, 5, 3, 3,
 0, 1, 2, 0, 3, 3, 3, 4, 1, 1, 0, 2, 3, 3, 1, 0, 1, 4, 4, 4, 4, 4, 1, 3, 1, 0, 1, 4, 1, 0, 1, 4, 1, 0, 4, 0, 4,
 0]

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(label==n*np.ones(len(label)))) if x > 0] )
    
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1  
mat2 = pd.concat(matTest).reset_index()
mat1=pd.read_csv('/kaggle/input/nk-wwdd/mat1.csv')
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_2c1',
     'total_volume_mean_0c1',
     'total_volume_mean_1c1', 
     'total_volume_mean_3c1',
     'total_volume_mean_4c1', 
     'total_volume_mean_2c1',
     'trade_size_mean_0c1',
     'trade_size_mean_1c1', 
     'trade_size_mean_3c1',
     'trade_size_mean_4c1', 
     'trade_size_mean_2c1',
     'trade_order_count_mean_0c1',
     'trade_order_count_mean_1c1',
     'trade_order_count_mean_3c1',
     'trade_order_count_mean_4c1',
     'trade_order_count_mean_2c1',      
     'price_spread_mean_0c1',
     'price_spread_mean_1c1',
     'price_spread_mean_3c1',
     'price_spread_mean_4c1',
     'price_spread_mean_2c1',   
     'bid_spread_mean_0c1',
     'bid_spread_mean_1c1',
     'bid_spread_mean_3c1',
     'bid_spread_mean_4c1',
     'bid_spread_mean_2c1',       
     'ask_spread_mean_0c1',
     'ask_spread_mean_1c1',
     'ask_spread_mean_3c1',
     'ask_spread_mean_4c1',
     'ask_spread_mean_2c1',   
     'volume_imbalance_mean_0c1',
     'volume_imbalance_mean_1c1',
     'volume_imbalance_mean_3c1',
     'volume_imbalance_mean_4c1',
     'volume_imbalance_mean_2c1',       
     'bid_ask_spread_mean_0c1',
     'bid_ask_spread_mean_1c1',
     'bid_ask_spread_mean_3c1',
     'bid_ask_spread_mean_4c1',
     'bid_ask_spread_mean_2c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_2c1'] 
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

[1, 11, 22, 50, 55, 56, 62, 73, 76, 78, 84, 87, 96, 101, 112, 116, 122, 124, 126]
[0, 4, 5, 10, 15, 16, 17, 23, 26, 28, 29, 36, 42, 44, 48, 53, 66, 69, 72, 85, 94, 95, 100, 102, 109, 111, 113, 115, 118, 120]
[3, 6, 9, 18, 61, 63, 86, 97]
[27, 31, 33, 37, 38, 40, 58, 59, 60, 74, 75, 77, 82, 83, 88, 89, 90, 98, 99, 110]
[2, 7, 13, 14, 19, 20, 21, 30, 32, 34, 35, 39, 41, 43, 46, 47, 51, 52, 64, 67, 68, 70, 93, 103, 104, 105, 107, 108, 114, 119, 123, 125]
[81]
[8, 80]


In [30]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
get_custom_objects().update({'swish': Activation(swish)})
hidden_units = (128,64,32)
stock_embedding_size = 24

train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
cat_data =train['stock_id']
del train
gc.collect()

def base_model():
    
    
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(538), name='num_data')


    
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='swish')(out)
        

   
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model

In [31]:
features_to_consider = list(test)
features_to_consider.remove('time_id')
features_to_consider.remove('row_id')
f_mean = np.load('/kaggle/input/nk-wwdd/f_mean.npy')
for i in range(len(features_to_consider)):
    test[features_to_consider[i]] = test[features_to_consider[i]].fillna(f_mean[i])

In [32]:
test.head()

,stock_id,time_id,row_id,wap1_mean,wap1_amax,wap1_std,wap2_mean,wap2_amax,wap2_std,log_return1_realized_volatility,...,bid_ask_spread_mean_0c1,bid_ask_spread_mean_1c1,bid_ask_spread_mean_3c1,bid_ask_spread_mean_4c1,bid_ask_spread_mean_2c1,size_tau2_0c1,size_tau2_1c1,size_tau2_3c1,size_tau2_4c1,size_tau2_2c1
0,0,4,0-4,0.250325,-0.543764,-2.429043,0.335747,-0.527097,-2.723074,-3.421779,...,-0.279017,0.823808,0.519245,-0.030445,0.358564,-0.086921,3.161571,0.590543,-0.161021,-0.106622
1,0,32,0-32,-0.001311,-0.000514,0.000281,-0.001294,-0.000760,0.000382,0.001459,...,-0.279017,-0.260539,0.519245,-0.030445,0.358564,-0.086921,-0.085898,0.590543,-0.161021,-0.106622
2,0,34,0-34,-0.001311,-0.000514,0.000281,-0.001294,-0.000760,0.000382,0.001459,...,-0.279017,-0.260539,0.519245,-0.030445,0.358564,-0.086921,-0.085898,0.590543,-0.161021,-0.106622


In [33]:
n_folds=5
test_preds = np.zeros(len(test))

for n_count in range(n_folds):
    
    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.004), 
        loss=root_mean_squared_per_error
    )
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    model.load_weights(f'../input/nk-wwdd/n1_model_{n_count}.hdf5')

    scaler=load(open(f'/kaggle/input/nk-wwdd/scaler_{n_count}.pkl', 'rb'))
    tt =scaler.transform(test[features_to_consider].values)
    test_preds += model.predict([test['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)
       
    features_to_consider.append('stock_id')
    del model
    gc.collect()

In [34]:
nk_ffgg1=test_preds/n_folds

In [35]:
!pip -q install ../input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.matlib

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

from scipy import stats
from scipy.stats import norm
from joblib import Parallel, delayed

import shutil
import glob

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts




plt.style.use('ggplot')
orange_black = [
    '#fdc029', '#df861d', '#FF6347', '#aa3d01', '#a30e15', '#800000', '#171820'
]
plt.rcParams['figure.figsize'] = (16,9)
plt.rcParams["figure.facecolor"] = '#FFFACD'
plt.rcParams["axes.facecolor"] = '#FFFFE0'
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = orange_black[3]
plt.rcParams["grid.alpha"] = 0.5
plt.rcParams["grid.linestyle"] = '--'


import warnings
warnings.filterwarnings("ignore")

In [37]:
data_dir = '../input/optiver-realized-volatility-prediction/'


def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap


def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap


def log_return(series):
    return np.log(series).diff()


def realized_volatility(series):
    return np.sqrt(np.sum(series**2))


def count_unique(series):
    return len(np.unique(series))


def read_train_test():
    
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    
    return test


def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
   
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)

    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    
    df['wap_mean'] = (df['wap1'] + df['wap2']) / 2
    
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])

    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    df['bas'] = (df[['ask_price1', 'ask_price2']].min(axis = 1)/ df[['bid_price1', 'bid_price2']].max(axis = 1) - 1) 
    df['h_spread_l1'] = df['ask_price1'] - df['bid_price1']
    df['h_spread_l2'] = df['ask_price2'] - df['bid_price2']
    df['log_return_bid_price1'] = np.log(df['bid_price1'].pct_change() + 1)
    df['log_return_ask_price1'] = np.log(df['ask_price1'].pct_change() + 1)
    df['log_return_bid_size1'] = np.log(df['bid_size1'].pct_change() + 1)
    df['log_return_ask_size1'] = np.log(df['ask_size1'].pct_change() + 1)
    df['log_ask_1_div_bid_1'] = np.log(df['ask_price1'] / df['bid_price1'])
    df['log_ask_1_div_bid_1_size'] = np.log(df['ask_size1'] / df['bid_size1'])
    df['log_return_bid_price2'] = np.log(df['bid_price2'].pct_change() + 1)
    df['log_return_ask_price2'] = np.log(df['ask_price2'].pct_change() + 1)
    df['log_return_bid_size2'] = np.log(df['bid_size2'].pct_change() + 1)
    df['log_return_ask_size2'] = np.log(df['ask_size2'].pct_change() + 1)
    df['log_ask_2_div_bid_2'] = np.log(df['ask_price2'] / df['bid_price2'])
    df['log_ask_2_div_bid_2_size'] = np.log(df['ask_size2'] / df['bid_size2'])
    
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    
    
    
    create_feature_dict = {
        'wap1': [np.mean,np.max,np.std],
        'wap2': [np.mean,np.max,np.std],
 
        'log_return1': [realized_volatility,np.sum],
        'log_return2': [realized_volatility,np.sum], 
        'log_return_bid_price1':[realized_volatility,np.sum],
        'log_return_ask_price1':[realized_volatility,np.sum],
        'log_return_bid_size1':[realized_volatility,np.sum],
        'log_return_ask_size1':[realized_volatility,np.sum], 
        'log_ask_1_div_bid_1':[realized_volatility,np.sum],
        'log_ask_1_div_bid_1_size':[realized_volatility,np.sum], 
        'log_return_bid_price2':[realized_volatility,np.sum], 
        'log_return_ask_price2':[realized_volatility,np.sum], 
        'log_return_bid_size2':[realized_volatility,np.sum],
        'log_return_ask_size2':[realized_volatility,np.sum], 
        'log_ask_2_div_bid_2':[realized_volatility,np.sum], 
        'log_ask_2_div_bid_2_size':[realized_volatility,np.sum], 
      
        'wap_balance': [np.mean,np.max,np.std,np.sum], 
        'wap_mean': [np.mean,np.max,np.std,np.sum],
        'price_spread':[np.mean,np.max,np.std,np.sum],
        'bid_spread':[np.mean,np.max,np.std,np.sum],
        'ask_spread':[np.mean,np.max,np.std,np.sum],
        'total_volume':[np.mean,np.max,np.std,np.sum],
        'volume_imbalance':[np.mean,np.max,np.std,np.sum],
        'bas':[np.mean,np.max,np.std,np.sum],
        'h_spread_l1':[np.mean,np.max,np.std,np.sum],
        'h_spread_l2':[np.mean,np.max,np.std,np.sum],
        'price_spread2':[np.mean,np.max,np.std,np.sum],
        "bid_ask_spread":[np.mean,np.max,np.std,np.sum],
    }
    
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
      
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
       
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
       
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    

    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')

  
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200'], axis = 1, inplace = True)
    
    
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
  
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max],
    }
    
   
    def get_stats_window(seconds_in_bucket, add_suffix = False):
       
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
      
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

  
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
     
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
      
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
  
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')

    
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200','time_id'], axis = 1, inplace = True)
    
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature


def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']


  
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
   
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
   
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
 
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    

def preprocessor(list_stock_ids, is_train = True):
    
    
    def for_joblib(stock_id):
       
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
       
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
 
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
       
        return df_tmp
    
  
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
   
    df = pd.concat(df, ignore_index = True)
    return df


def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

test = read_train_test()

 
test_stock_ids = test['stock_id'].unique()

test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')


test = get_time_stock(test)

test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )


test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished


In [38]:
from sklearn.cluster import KMeans


train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

label=[1, 0, 4, 2, 1, 1, 2, 4, 6, 2, 1, 0, 4, 4, 1, 1, 1, 2, 4, 4, 4, 0, 1, 1, 3, 1, 1, 4, 3, 4, 3, 4, 4, 1, 3, 3, 4,
 3, 4, 1, 4, 1, 4, 4, 1, 0, 4, 4, 1, 0, 0, 3, 3, 3, 2, 0, 2, 4, 1, 4, 4, 1, 4, 1, 0, 3, 3, 0, 3, 0, 6, 5, 3, 3,
 0, 1, 2, 0, 3, 3, 3, 4, 1, 1, 0, 2, 3, 3, 1, 0, 1, 4, 4, 4, 4, 4, 1, 3, 1, 0, 1, 4, 1, 0, 1, 4, 1, 0, 4, 0, 4,
 0]

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(label==n*np.ones(len(label)))) if x > 0] )
    


matTest = []

n = 0
for ind in l:
    print(ind)
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1

mat2 = pd.concat(matTest).reset_index()
mat1=pd.read_csv('/kaggle/input/tk-wwdd/mat1.csv')
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_2c1',
     'total_volume_mean_0c1',
     'total_volume_mean_1c1', 
     'total_volume_mean_3c1',
     'total_volume_mean_4c1', 
     'total_volume_mean_2c1',
     'trade_size_mean_0c1',
     'trade_size_mean_1c1', 
     'trade_size_mean_3c1',
     'trade_size_mean_4c1', 
     'trade_size_mean_2c1',
     'trade_order_count_mean_0c1',
     'trade_order_count_mean_1c1',
     'trade_order_count_mean_3c1',
     'trade_order_count_mean_4c1',
     'trade_order_count_mean_2c1',      
     'price_spread_mean_0c1',
     'price_spread_mean_1c1',
     'price_spread_mean_3c1',
     'price_spread_mean_4c1',
     'price_spread_mean_2c1',   
     'bid_spread_mean_0c1',
     'bid_spread_mean_1c1',
     'bid_spread_mean_3c1',
     'bid_spread_mean_4c1',
     'bid_spread_mean_2c1',       
     'ask_spread_mean_0c1',
     'ask_spread_mean_1c1',
     'ask_spread_mean_3c1',
     'ask_spread_mean_4c1',
     'ask_spread_mean_2c1',   
     'volume_imbalance_mean_0c1',
     'volume_imbalance_mean_1c1',
     'volume_imbalance_mean_3c1',
     'volume_imbalance_mean_4c1',
     'volume_imbalance_mean_2c1',       
     'bid_ask_spread_mean_0c1',
     'bid_ask_spread_mean_1c1',
     'bid_ask_spread_mean_3c1',
     'bid_ask_spread_mean_4c1',
     'bid_ask_spread_mean_2c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_2c1'] 
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

[1, 11, 22, 50, 55, 56, 62, 73, 76, 78, 84, 87, 96, 101, 112, 116, 122, 124, 126]
[0, 4, 5, 10, 15, 16, 17, 23, 26, 28, 29, 36, 42, 44, 48, 53, 66, 69, 72, 85, 94, 95, 100, 102, 109, 111, 113, 115, 118, 120]
[3, 6, 9, 18, 61, 63, 86, 97]
[27, 31, 33, 37, 38, 40, 58, 59, 60, 74, 75, 77, 82, 83, 88, 89, 90, 98, 99, 110]
[2, 7, 13, 14, 19, 20, 21, 30, 32, 34, 35, 39, 41, 43, 46, 47, 51, 52, 64, 67, 68, 70, 93, 103, 104, 105, 107, 108, 114, 119, 123, 125]
[81]
[8, 80]


In [39]:
features = [col for col in test.columns.tolist() if col not in ['time_id','row_id']]
f_mean = np.load('/kaggle/input/tk-wwdd/f_mean.npy')
for i in range(len(features)):
    test[features[i]] = test[features[i]].fillna(f_mean[i])

In [40]:
X_test=test.copy()
X_test.drop(['time_id','row_id'], axis=1,inplace=True)
def rmspe(y_true, y_pred):
    
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

class RMSPE(Metric):
    def __init__(self):
        self._name = "rmspe"
        self._maximize = False

    def __call__(self, y_true, y_score):
        
        return np.sqrt(np.mean(np.square((y_true - y_score) / y_true)))
    


def RMSPELoss(y_pred, y_true):
    return torch.sqrt(torch.mean( ((y_true - y_pred) / y_true) ** 2 )).clone()
from pickle import load,dump

for col in features:
    if  col == 'stock_id':
        l_enc=load(open('/kaggle/input/tk-wwdd/l_enc.pkl', 'rb'))
        X_test[col] = l_enc.transform(X_test[col].values)
    else:
        scaler=load(open(f'/kaggle/input/tk-wwdd/scaler_{col}.pkl', 'rb'))
        X_test[col] = scaler.transform(X_test[col].values.reshape(-1, 1))
cat_idxs_df=pd.read_csv('/kaggle/input/tk-wwdd/cat_idxs_df.csv')
cat_dims_df=pd.read_csv('/kaggle/input/tk-wwdd/cat_dims_df.csv')
cat_idxs=cat_idxs_df['cat_idxs']
cat_dims=cat_dims_df['cat_dims']
tabnet_params = dict(
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=1,
    n_d = 13,
    n_a = 13,
    n_steps = 1,
    gamma = 2,
    n_independent = 2,
    n_shared = 2,
    lambda_sparse = 0,
    optimizer_fn = Adam,
    optimizer_params = dict(lr = (2e-2)),
    mask_type = "entmax",
    scheduler_params = dict(T_0=200, T_mult=1, eta_min=1e-4, last_epoch=-1, verbose=False),
    scheduler_fn = CosineAnnealingWarmRestarts,
    seed = 1,
    verbose = 10
    
)

In [41]:
X_test.head()

,stock_id,wap1_mean,wap1_amax,wap1_std,wap2_mean,wap2_amax,wap2_std,log_return1_realized_volatility,log_return1_sum,log_return2_realized_volatility,...,bid_ask_spread_mean_0c1,bid_ask_spread_mean_1c1,bid_ask_spread_mean_3c1,bid_ask_spread_mean_4c1,bid_ask_spread_mean_2c1,size_tau2_0c1,size_tau2_1c1,size_tau2_3c1,size_tau2_4c1,size_tau2_2c1
0,0,1.191565e-01,-4.375152e-01,-8.941496e-01,1.623597e-01,-4.365765e-01,-9.351345e-01,-1.098426e+00,8.241560e-02,-1.105520e+00,...,1.248494e-13,1.434182e+00,4.230394e-13,2.611967e-13,-1.809262e-13,-2.844108e-13,2.003125e+01,-5.454171e-13,-1.019213e-12,-1.264623e-13
1,0,-2.042257e-12,1.225712e-12,3.505232e-15,9.872247e-13,-5.769340e-14,9.161438e-15,9.433118e-15,4.194278e-17,9.664863e-15,...,1.248494e-13,2.431447e-13,4.230394e-13,2.611967e-13,-1.809262e-13,-2.844108e-13,6.856557e-13,-5.454171e-13,-1.019213e-12,-1.264623e-13
2,0,-2.042257e-12,1.225712e-12,3.505232e-15,9.872247e-13,-5.769340e-14,9.161438e-15,9.433118e-15,4.194278e-17,9.664863e-15,...,1.248494e-13,2.431447e-13,4.230394e-13,2.611967e-13,-1.809262e-13,-2.844108e-13,6.856557e-13,-5.454171e-13,-1.019213e-12,-1.264623e-13


In [42]:
import os
pathB = "../input/tk-wwdd/"
modelpath = [os.path.join(pathB,s) for s in os.listdir(pathB) if ("zip" in s)]

In [43]:
test_predictions=[]
for path in modelpath:
    clf =  TabNetRegressor(**tabnet_params)
    clf.load_model(path)
    test_predictions.append(clf.predict(X_test.values).squeeze(-1))

Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda
Device used : cuda


In [44]:
tk_ffgg1 =  np.mean(test_predictions,axis=0)

In [45]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np 
import pandas as pd 
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib

target_name = 'target'

import joblib

In [46]:
data_dir = '../input/optiver-realized-volatility-prediction/'

def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap


def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap


def log_return(series):
    return np.log(series).diff()


def realized_volatility(series):
    return np.sqrt(np.sum(series**2))


def count_unique(series):
    return len(np.unique(series))


def read_test():
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
   
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    return test


def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)

    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)
 
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)

    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
   
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
   
    create_feature_dict = {
        'wap1': [np.sum, np.std],
        'wap2': [np.sum, np.std],
        'wap3': [np.sum, np.std],
        'wap4': [np.sum, np.std],
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'wap_balance': [np.sum, np.max],
        'price_spread':[np.sum, np.max],
        'price_spread2':[np.sum, np.max],
        'bid_spread':[np.sum, np.max],
        'ask_spread':[np.sum, np.max],
        'total_volume':[np.sum, np.max],
        'volume_imbalance':[np.sum, np.max],
        "bid_ask_spread":[np.sum,  np.max],
    }
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
    }
    
 
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
    
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
      
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
  
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
   
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

 
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
   
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    
 
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature


def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount']=df['price']*df['size']
   
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
    }
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
    }
 
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
       
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
       
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
   
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    


    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
  
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        

        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
  
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
   
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature


def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']


  
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
  
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

 
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
  
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    

    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    

def preprocessor(list_stock_ids, is_train = True):
    
  
    def for_joblib(stock_id):
 
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
    
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
     
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
       
        return df_tmp
    
  
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
  
    df = pd.concat(df, ignore_index = True)
    return df


def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

test = read_test()


test_stock_ids = test['stock_id'].unique()

test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')


test = get_time_stock(test)

test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )


test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [47]:
from sklearn.cluster import KMeans
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

label=[1, 0, 4, 2, 1, 1, 2, 4, 6, 2, 1, 0, 4, 4, 1, 1, 1, 2, 4, 4, 4, 0, 1, 1, 3, 1, 1, 4, 3, 4, 3, 4, 4, 1, 3, 3, 4,
 3, 4, 1, 4, 1, 4, 4, 1, 0, 4, 4, 1, 0, 0, 3, 3, 3, 2, 0, 2, 4, 1, 4, 4, 1, 4, 1, 0, 3, 3, 0, 3, 0, 6, 5, 3, 3,
 0, 1, 2, 0, 3, 3, 3, 4, 1, 1, 0, 2, 3, 3, 1, 0, 1, 4, 4, 4, 4, 4, 1, 3, 1, 0, 1, 4, 1, 0, 1, 4, 1, 0, 4, 0, 4,
 0]

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(label==n*np.ones(len(label)))) if x > 0] )
    
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1  
mat2 = pd.concat(matTest).reset_index()
mat1=pd.read_csv('/kaggle/input/lk-wwdd/mat1.csv')
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_2c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_2c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_2c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_2c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_2c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_2c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_2c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_2c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_2c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_2c1'] 
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

[1, 11, 22, 50, 55, 56, 62, 73, 76, 78, 84, 87, 96, 101, 112, 116, 122, 124, 126]
[0, 4, 5, 10, 15, 16, 17, 23, 26, 28, 29, 36, 42, 44, 48, 53, 66, 69, 72, 85, 94, 95, 100, 102, 109, 111, 113, 115, 118, 120]
[3, 6, 9, 18, 61, 63, 86, 97]
[27, 31, 33, 37, 38, 40, 58, 59, 60, 74, 75, 77, 82, 83, 88, 89, 90, 98, 99, 110]
[2, 7, 13, 14, 19, 20, 21, 30, 32, 34, 35, 39, 41, 43, 46, 47, 51, 52, 64, 67, 68, 70, 93, 103, 104, 105, 107, 108, 114, 119, 123, 125]
[81]
[8, 80]


In [48]:
test.head()

,stock_id,time_id,row_id,wap1_sum,wap1_std,wap2_sum,wap2_std,wap3_sum,wap3_std,wap4_sum,...,bid_ask_spread_sum_0c1,bid_ask_spread_sum_1c1,bid_ask_spread_sum_3c1,bid_ask_spread_sum_4c1,bid_ask_spread_sum_2c1,size_tau2_0c1,size_tau2_1c1,size_tau2_3c1,size_tau2_4c1,size_tau2_2c1
0,0,4,0-4,3.001215,0.00017,3.00165,0.000153,3.000752,0.000142,2.999481,...,NaN,0.001524,NaN,NaN,NaN,NaN,0.301511,NaN,NaN,NaN
1,0,32,0-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,34,0-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
from sklearn.model_selection import KFold
import lightgbm as lgb
import glob
import joblib
import lightgbm as lgb
x_test = test.drop(['row_id', 'time_id'], axis = 1)
x_test['stock_id'] = x_test['stock_id'].astype(int)

In [50]:
MODEL_DIR =  "../input/lk-wwdd"
lk_ffgg1 = np.zeros(len(x_test))
files = glob.glob(f'{MODEL_DIR}/*model*.pkl')
for i, f in enumerate(files):
    model = joblib.load(f)
    lk_ffgg1 += model.predict(x_test)
lk_ffgg1=lk_ffgg1/5

In [51]:
print(ng_ffgg1)
print(tg_ffgg1)
print(lg_ffgg1)
print(nk_ffgg1)
print(tk_ffgg1)
print(lk_ffgg1)

[0.00114634 0.00251188 0.00251188]
[0.00241323 0.00311746 0.00311746]
[0.00094421 0.00098578 0.00098578]
[0.00078814 0.00242929 0.00242929]
[0.00195556 0.00308201 0.00308201]
[0.0014088  0.00149299 0.00149299]


In [52]:
test['target'] = np.median([ng_ffgg1,tg_ffgg1,lg_ffgg1,nk_ffgg1,tk_ffgg1,lk_ffgg1],axis=0)
test[['row_id', 'target']].to_csv('submission.csv', index = False)

In [53]:
test[['row_id', 'target']].head()

,row_id,target
0,0-4,0.001278
1,0-32,0.002471
2,0-34,0.002471
